In [6]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
import numpy as np
import matplotlib.pyplot as plt

In [7]:
IMG_SIZE = 32
SEED = 1000
BATCH_SIZE = 32
NUM_CLASSES = 10
EPOCHS = 100

In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 6s 0us/step


In [9]:
y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

In [10]:
train_gen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2,
    rescale=1./255
)

In [11]:
test_gen = ImageDataGenerator(rescale=1./255)

In [12]:
train_data = train_gen.flow(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    seed=SEED,
    subset='training'
)


In [13]:
valid_data = train_gen.flow(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    seed=SEED,
    subset='validation'
)

In [14]:
test_data = test_gen.flow(
    x_test,
    y_test,
    batch_size=BATCH_SIZE,
    seed=SEED
)

In [15]:
base_model = tf.keras.applications.VGG16(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

58889256/58889256 [==============================] - 2s 0us/step


In [16]:
for layer in base_model.layers:
    layer.trainable = False

In [17]:
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

In [18]:
model = tf.keras.Model(inputs, output)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

In [19]:
history_frozen = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=valid_data,
    callbacks=[
        EarlyStopping(patience=5),
        LearningRateScheduler(lambda epoch: 1e-3 * 0.95 ** epoch)
    ]
)

Epoch 1/100
1250/1250 [==============================] - 53s 34ms/step - loss: 1.5868 - accuracy: 0.4374 - val_loss: 1.3904 - val_accuracy: 0.5070 - lr: 0.0010
Epoch 2/100
1250/1250 [==============================] - 42s 34ms/step - loss: 1.3986 - accuracy: 0.5095 - val_loss: 1.3343 - val_accuracy: 0.5318 - lr: 9.5000e-04
Epoch 3/100
1250/1250 [==============================] - 41s 33ms/step - loss: 1.3479 - accuracy: 0.5300 - val_loss: 1.2777 - val_accuracy: 0.5492 - lr: 9.0250e-04
Epoch 4/100
1250/1250 [==============================] - 41s 33ms/step - loss: 1.3026 - accuracy: 0.5456 - val_loss: 1.2470 - val_accuracy: 0.5622 - lr: 8.5737e-04
Epoch 5/100
1250/1250 [==============================] - 44s 35ms/step - loss: 1.2706 - accuracy: 0.5567 - val_loss: 1.2279 - val_accuracy: 0.5693 - lr: 8.1451e-04
Epoch 6/100
 787/1250 [=================>............] - ETA: 12s - loss: 1.2516 - accuracy: 0.5637

KeyboardInterrupt: ignored

In [20]:
for layer in base_model.layers:
    layer.trainable = True

In [21]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=['accuracy']
)

In [22]:
history_unfrozen = model.fit(
    train_data,
    epochs=EPOCHS,
    validation_data=valid_data,
    callbacks=[
        EarlyStopping(patience=5),
        LearningRateScheduler(lambda epoch: 1e-4 * 0.95 ** epoch)
    ]
)

Epoch 1/100
1250/1250 [==============================] - 57s 39ms/step - loss: 1.0780 - accuracy: 0.6352 - val_loss: 0.8976 - val_accuracy: 0.6942 - lr: 1.0000e-04
Epoch 2/100
1250/1250 [==============================] - 50s 40ms/step - loss: 0.7444 - accuracy: 0.7528 - val_loss: 0.7222 - val_accuracy: 0.7505 - lr: 9.5000e-05
Epoch 3/100
1250/1250 [==============================] - 49s 39ms/step - loss: 0.6309 - accuracy: 0.7924 - val_loss: 0.6791 - val_accuracy: 0.7821 - lr: 9.0250e-05
Epoch 4/100
1250/1250 [==============================] - 49s 39ms/step - loss: 0.5442 - accuracy: 0.8227 - val_loss: 0.5541 - val_accuracy: 0.8172 - lr: 8.5737e-05
Epoch 5/100
1250/1250 [==============================] - 49s 39ms/step - loss: 0.4878 - accuracy: 0.8397 - val_loss: 0.5515 - val_accuracy: 0.8193 - lr: 8.1451e-05
Epoch 6/100
 176/1250 [===>..........................] - ETA: 39s - loss: 0.4252 - accuracy: 0.8620

KeyboardInterrupt: ignored

In [24]:
# Plot the training and validation accuracy for both cases
plt.plot(history_frozen.history['accuracy'], label='Frozen base layers')
plt.plot(history_unfrozen.history['accuracy'], label='Unfrozen base layers')
plt.title('Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot the training and validation loss for both cases
plt.plot(history_frozen.history['loss'], label='Frozen base layers')
plt.plot(history_unfrozen.history['loss'], label='Unfrozen base layers')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


NameError: ignored